# 🧠 Brain Tumor Segmentation Using FFT Filters
## BME 271D Final Project - Ege Özemek, Max Bazan, Sasha Nikiforov

### Frequency-Domain Analysis for Medical Image Segmentation

**This notebook demonstrates:**
- FFT-based tumor segmentation (High-Pass & Band-Pass filters)
- Comparison with spatial domain methods (Otsu, Blob detection)
- Quantitative evaluation (Dice, IoU, Boundary Accuracy)
- Multi-tier confidence scoring for clinical decision support

---
## 📚 Project Overview & Motivation

### Clinical Context
Brain tumor segmentation is a critical step in:
- **Surgical Planning**: Precise tumor boundaries guide neurosurgical procedures
- **Treatment Monitoring**: Tracking tumor size changes during therapy
- **Radiotherapy Planning**: Targeting radiation to tumor while sparing healthy tissue
- **Diagnostic Workflow**: Reducing radiologist workload through automation

### Why Frequency-Domain Methods?
Traditional spatial-domain segmentation (like Otsu thresholding) works well on high-contrast images but struggles with:
- **Complex texture patterns** within tumors
- **Subtle intensity variations** between tumor and healthy tissue
- **Noise and artifacts** in medical imaging

**Fourier Transform** decomposes images into frequency components:
- **Low frequencies** = smooth backgrounds, large structures
- **High frequencies** = edges, boundaries, fine details
- **Mid frequencies** = texture patterns

By filtering in the frequency domain, we can:

✅ **Emphasize tumor boundaries** (high-pass filtering)  
✅ **Isolate tumor texture** (band-pass filtering)  
✅ **Suppress background noise** (complementary filtering)

### Course Concepts Applied (BME 271D)
This project demonstrates key concepts from Signals & Systems:
1. **2D Fourier Transform**: Extending 1D FFT to images
2. **Filter Design**: High-pass, band-pass, and low-pass filters
3. **Frequency Response**: Understanding how filters affect different frequency components
4. **Convolution**: Morphological operations as convolution with structuring elements
5. **System Analysis**: Comparing multiple methods quantitatively

---

## 🎯 Workflow Summary

```
1. Upload MRI Image → Load and normalize to [0,1]
2. Upload Ground Truth (Optional) → For quantitative evaluation
3. FFT Analysis → Visualize frequency spectrum
4. Apply Filters → High-pass, Band-pass, Combined
5. Segment Tumor → 6 different methods
6. Calculate Metrics → Dice, IoU, Boundary Accuracy
7. Generate Report → Clinical confidence scoring
```

**Expected Runtime**: 2-5 minutes per image

**Best Results With**: T1-weighted contrast-enhanced MRI (bright tumors work best)

---

## 🔧 Setup: Installation & Imports

This cell installs all required packages and imports the necessary libraries.

**Key Libraries:**
- `numpy`: Numerical operations and array handling
- `scipy`: FFT computation and signal processing
- `scikit-image`: Image processing and morphological operations
- `matplotlib`: Visualization
- `pandas`: Data organization for metrics

**Custom Module:**
- `tumor_segmentation.py`: Contains core FFT filtering functions developed for this project

In [ ]:
# ========== INSTALLATION & IMPORTS ==========
!pip install -q numpy matplotlib scipy scikit-image pandas
!wget -q https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/tumor_segmentation.py

import tumor_segmentation as ts
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image as PILImage
from google.colab import files
from scipy import ndimage
from scipy.spatial import distance
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects, binary_closing, binary_opening, binary_dilation, disk, erosion
from skimage.segmentation import find_boundaries

print('✅ Setup complete!')

# ========== HELPER FUNCTIONS ==========

def load_image_safe(filepath):
    """Load image and convert to grayscale float [0,1]"""
    img = PILImage.open(filepath).convert('L')
    arr = np.array(img).astype(np.float64) / 255.0
    return arr

def load_mask_safe(filepath):
    """Load mask and convert to binary [0,1]"""
    img = PILImage.open(filepath).convert('L')
    arr = np.array(img)
    # Convert from 0-255 to 0-1
    return (arr > 127).astype(np.float32)

def remove_peripheral(mask, image, border_frac=0.12):
    """Remove skull and peripheral structures from segmentation."""
    h, w = mask.shape
    border_h, border_w = int(h * border_frac), int(w * border_frac)

    # Define internal region (excluding borders)
    internal = np.zeros_like(mask, dtype=bool)
    internal[border_h:-border_h, border_w:-border_w] = True

    # Filter regions
    labeled = label(mask)
    cleaned = np.zeros_like(mask, dtype=bool)

    for region in regionprops(labeled):
        region_pixels = (labeled == region.label)
        internal_pixels = np.logical_and(region_pixels, internal).sum()

        # Region must be >60% internal (not on border)
        if internal_pixels / region.area > 0.6:
            cleaned[region_pixels] = True

    return cleaned

# ========== SEGMENTATION METHODS ==========

def otsu_baseline(image):
    """Otsu thresholding baseline method."""
    try:
        thresh = threshold_otsu(image)
        mask = image > thresh
        mask = binary_closing(mask, disk(2))
        mask = remove_small_objects(mask, min_size=100)
        mask = remove_peripheral(mask, image)
        return mask.astype(np.float32)
    except:
        return np.zeros_like(image, dtype=np.float32)

def fft_highpass(image, sensitivity=0.5):
    """FFT High-Pass filter segmentation."""
    try:
        mean_val, std_val = image.mean(), image.std()

        # Apply high-pass filter
        hp_img, _, _ = ts.filter_pipeline(image, 'hp', cutoff_radius=25)
        hp_img = (hp_img - hp_img.min()) / (hp_img.max() - hp_img.min() + 1e-8)

        # Threshold
        hp_mask = hp_img > np.percentile(hp_img, 87 - sensitivity * 7)

        # Must be bright in original
        hp_mask = np.logical_and(hp_mask, image > mean_val + 0.5 * std_val)

        # Morphological cleanup
        hp_mask = binary_closing(hp_mask, disk(2))
        hp_mask = remove_small_objects(hp_mask, min_size=80)
        hp_mask = remove_peripheral(hp_mask, image)

        return hp_mask.astype(np.float32)
    except:
        return np.zeros_like(image, dtype=np.float32)

def fft_bandpass(image, sensitivity=0.5):
    """FFT Band-Pass filter segmentation."""
    try:
        mean_val, std_val = image.mean(), image.std()

        # Apply band-pass filter
        bp_img, _, _ = ts.filter_pipeline(image, 'bp', r1=10, r2=50)
        bp_img = (bp_img - bp_img.min()) / (bp_img.max() - bp_img.min() + 1e-8)

        # Threshold
        bp_mask = bp_img > np.percentile(bp_img, 83 - sensitivity * 7)

        # Must be bright in original
        bp_mask = np.logical_and(bp_mask, image > mean_val + 0.5 * std_val)

        # Morphological cleanup
        bp_mask = binary_closing(bp_mask, disk(2))
        bp_mask = remove_small_objects(bp_mask, min_size=80)
        bp_mask = remove_peripheral(bp_mask, image)

        return bp_mask.astype(np.float32)
    except:
        return np.zeros_like(image, dtype=np.float32)

def fft_combined(image, sensitivity=0.5):
    """Combined FFT method (OR of high-pass and band-pass)."""
    hp = fft_highpass(image, sensitivity)
    bp = fft_bandpass(image, sensitivity)
    combined = np.logical_or(hp, bp).astype(np.float32)
    return combined

def blob_detection(image, sensitivity=0.5):
    """Blob detection for bright tumor masses."""
    try:
        h, w = image.shape
        mean_val, std_val = image.mean(), image.std()

        # Find very bright regions
        bright_thresh = mean_val + (2.2 - sensitivity * 0.4) * std_val
        percentile_thresh = np.percentile(image, 92 - sensitivity * 5)

        blob_mask = np.logical_and(image > bright_thresh, image > percentile_thresh)

        # Morphological operations
        blob_mask = binary_opening(blob_mask, disk(2))
        blob_mask = binary_closing(blob_mask, disk(3))
        blob_mask = remove_small_objects(blob_mask, min_size=120)
        blob_mask = remove_peripheral(blob_mask, image)

        # Shape filtering
        labeled = label(blob_mask)
        filtered = np.zeros_like(blob_mask)

        for region in regionprops(labeled, intensity_image=image):
            # Size constraints
            if region.area < 100 or region.area > h * w * 0.25:
                continue

            # Shape constraints
            circularity = 4 * np.pi * region.area / (region.perimeter ** 2 + 1e-8)
            if circularity < 0.12:
                continue
            if region.solidity < 0.65:
                continue

            # Intensity constraint
            if region.mean_intensity < mean_val + std_val:
                continue

            filtered[labeled == region.label] = True

        return filtered.astype(np.float32)
    except:
        return np.zeros_like(image, dtype=np.float32)

def hybrid_combined(image, sensitivity=0.5):
    """Hybrid ensemble combining FFT and blob detection."""
    fft_comb = fft_combined(image, sensitivity)
    blob = blob_detection(image, sensitivity)

    # Voting: FFT gets 1.5 votes, Blob gets 1 vote
    vote_map = fft_comb * 1.5 + blob * 1.0

    # Need 1.0+ votes (any single method can trigger)
    combined = (vote_map >= (1.0 - sensitivity * 0.1)).astype(np.float32)

    # Final cleanup
    combined = binary_closing(combined.astype(bool), disk(2))
    combined = remove_small_objects(combined, min_size=100)

    # Shape filter
    labeled = label(combined)
    final = np.zeros_like(combined)

    for region in regionprops(labeled):
        if region.eccentricity > 0.96 or region.solidity < 0.65:
            continue
        final[labeled == region.label] = True

    return final.astype(np.float32)

# ========== ANALYSIS FUNCTIONS ==========

def get_tumor_center(mask):
    """Get tumor center using largest connected component centroid."""
    if mask.sum() == 0:
        return None

    # Find largest connected component
    labeled = label(mask)
    regions = regionprops(labeled)

    if len(regions) == 0:
        return None

    # Get centroid of largest region
    largest = max(regions, key=lambda x: x.area)
    centroid = largest.centroid  # (row, col)

    return (int(centroid[0]), int(centroid[1]))

def calculate_confidence(image, mask):
    """Calculate detection confidence with 3-tier system."""
    h, w = image.shape
    total = h * w
    tumor_px = mask.sum()
    area_pct = (tumor_px / total) * 100

    # No detection if too small
    if tumor_px < total * 0.003:  # <0.3%
        return 0.0, 'NONE'

    labeled = label(mask)
    regions = regionprops(labeled, intensity_image=image)

    if len(regions) == 0:
        return 0.0, 'NONE'

    largest = max(regions, key=lambda x: x.area)

    # Size score
    if 0.4 <= area_pct <= 12:
        size_score = 1.0
    elif area_pct < 0.4:
        size_score = area_pct / 0.4
    else:
        size_score = max(0, 1 - (area_pct - 12) / 18)

    # Shape score
    circularity = 4 * np.pi * largest.area / (largest.perimeter ** 2 + 1e-8)
    shape_score = min(circularity * 2.5, 1.0)

    # Contrast score
    tumor_int = image[mask > 0].mean()
    bg_int = image[mask == 0].mean() if (mask == 0).any() else 0
    contrast_score = min(max((tumor_int - bg_int) * 4, 0), 1.0)

    # Overall confidence
    confidence = size_score * 0.3 + shape_score * 0.3 + contrast_score * 0.4
    confidence = np.clip(confidence, 0, 1)

    # Determine tier
    if confidence >= 0.65:
        tier = 'HIGH'
    elif confidence >= 0.35:
        tier = 'MODERATE'
    elif confidence >= 0.20:
        tier = 'LOW'
    else:
        tier = 'NONE'

    return confidence, tier

# ========== QUANTITATIVE METRICS ==========

def dice_coefficient(pred, true):
    """Calculate Dice Similarity Coefficient."""
    pred = (pred > 0.5).astype(np.float32)
    true = (true > 0.5).astype(np.float32)

    intersection = np.sum(pred * true)
    pred_sum = np.sum(pred)
    true_sum = np.sum(true)

    if pred_sum + true_sum == 0:
        return 1.0  # Both empty

    dice = (2.0 * intersection) / (pred_sum + true_sum)
    return dice

def iou_score(pred, true):
    """Calculate Intersection over Union."""
    pred = (pred > 0.5).astype(np.float32)
    true = (true > 0.5).astype(np.float32)

    intersection = np.sum(pred * true)
    union = np.sum(np.logical_or(pred, true))

    if union == 0:
        return 1.0  # Both empty

    iou = intersection / union
    return iou

def boundary_accuracy(pred, true, tolerance=2):
    """Calculate boundary accuracy within tolerance (pixels)."""
    pred = (pred > 0.5).astype(bool)
    true = (true > 0.5).astype(bool)

    # Find boundaries
    pred_boundary = find_boundaries(pred, mode='inner')
    true_boundary = find_boundaries(true, mode='inner')

    if not pred_boundary.any() or not true_boundary.any():
        return 0.0

    # Get boundary coordinates
    pred_coords = np.argwhere(pred_boundary)
    true_coords = np.argwhere(true_boundary)

    if len(pred_coords) == 0 or len(true_coords) == 0:
        return 0.0

    # For each predicted boundary pixel, find distance to nearest true boundary pixel
    distances = distance.cdist(pred_coords, true_coords, metric='euclidean')
    min_distances = distances.min(axis=1)

    # Calculate percentage within tolerance
    within_tolerance = (min_distances <= tolerance).sum()
    accuracy = within_tolerance / len(pred_coords)

    return accuracy

def evaluate_all_methods(image, ground_truth, sensitivity=0.5):
    """Evaluate all segmentation methods against ground truth."""
    methods = {
        'Otsu Baseline': otsu_baseline(image),
        'FFT High-Pass': fft_highpass(image, sensitivity),
        'FFT Band-Pass': fft_bandpass(image, sensitivity),
        'FFT Combined': fft_combined(image, sensitivity),
        'Blob Detection': blob_detection(image, sensitivity),
        'Hybrid Combined': hybrid_combined(image, sensitivity)
    }

    results = []

    for method_name, pred_mask in methods.items():
        dice = dice_coefficient(pred_mask, ground_truth)
        iou = iou_score(pred_mask, ground_truth)
        boundary_acc = boundary_accuracy(pred_mask, ground_truth, tolerance=2)

        results.append({
            'Method': method_name,
            'Dice': dice,
            'IoU': iou,
            'Boundary Accuracy': boundary_acc
        })

    return pd.DataFrame(results), methods

# Initialize variables
mri_image = None
ground_truth_mask = None

print('\n✅ All functions loaded successfully!')
print('📋 Ready to process images!')

---
## 📤 Step 1: Upload MRI Image (REQUIRED)

Upload your brain MRI scan here.

### Image Requirements:
- **Format**: JPEG, PNG, or other common image formats
- **Modality**: Works best with T1-weighted contrast-enhanced MRI
- **Quality**: Higher resolution = better results
- **Content**: Should show axial brain slice with visible tumor

### What Happens:
1. Image is loaded and converted to grayscale
2. Pixel values are normalized to [0, 1] range
3. Image is displayed for verification

**Note**: This is the only required upload. The ground truth mask (Step 2) is optional.

In [ ]:
# Upload MRI Image
print('📤 Please upload your MRI image...')
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    mri_image = load_image_safe(filename)

    print(f'\n✅ MRI Image loaded: {filename}')
    print(f'   Size: {mri_image.shape}')
    print(f'   Range: [{mri_image.min():.3f}, {mri_image.max():.3f}]')

    # Display
    plt.figure(figsize=(8, 8))
    plt.imshow(mri_image, cmap='gray')
    plt.title('Uploaded MRI Image', fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print('❌ No image uploaded!')

---
## 📤 Step 2: Upload Ground Truth Mask (OPTIONAL)

**Skip this step if you don't have a ground truth mask.**

If you have a mask, upload it here to get quantitative evaluation metrics.

### Mask Requirements:
- **Format**: PNG (binary mask)
- **Size**: Must exactly match MRI image dimensions
- **Values**:
  - White (255) = Tumor region
  - Black (0) = Background
- **Purpose**: Enables calculation of Dice, IoU, and Boundary Accuracy

### What You Get With a Mask:
✅ **Quantitative Metrics**: Dice coefficient, IoU, Boundary accuracy  
✅ **Method Comparison**: Bar plots showing which methods perform best  
✅ **Performance Analysis**: Statistical comparison to Otsu baseline

### What You Get Without a Mask:
- Visual segmentation results
- Confidence scores
- Tumor localization (green cross)
- Clinical report

**Note**: For testing purposes, you can skip this and still see all detection results.

In [ ]:
# Upload Ground Truth Mask (Optional)
print('📤 Upload ground truth mask (or skip if you don\'t have one)...')
uploaded_mask = files.upload()

if uploaded_mask:
    mask_filename = list(uploaded_mask.keys())[0]
    ground_truth_mask = load_mask_safe(mask_filename)

    print(f'\n✅ Ground Truth Mask loaded: {mask_filename}')
    print(f'   Size: {ground_truth_mask.shape}')
    print(f'   Unique values: {np.unique(ground_truth_mask)}')
    print(f'   Tumor coverage: {ground_truth_mask.sum() / ground_truth_mask.size * 100:.2f}%')

    # Display
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    axes[0].imshow(mri_image, cmap='gray')
    axes[0].set_title('MRI Image', fontsize=12)
    axes[0].axis('off')

    axes[1].imshow(ground_truth_mask, cmap='gray')
    axes[1].set_title('Ground Truth Mask', fontsize=12)
    axes[1].axis('off')

    axes[2].imshow(mri_image, cmap='gray')
    axes[2].imshow(ground_truth_mask, cmap='Reds', alpha=0.5)
    axes[2].set_title('Overlay', fontsize=12)
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()
else:
    ground_truth_mask = None
    print('\nℹ️  No mask uploaded - will skip quantitative evaluation.')

---
## 🔬 Step 3: FFT Analysis

Visualize the frequency domain representation of the MRI.

### Understanding the FFT Spectrum

The **2D Fourier Transform** decomposes the image into sinusoidal components of different frequencies and orientations.

**In the frequency spectrum:**
- **Center (DC component)**: Average brightness of the image
- **Low frequencies (near center)**: Smooth variations, large structures, background
- **High frequencies (edges)**: Sharp transitions, boundaries, fine details
- **Tumor boundaries**: Appear as high-frequency components (bright edges)

### Course Connection (BME 271D)
This extends the 1D Fourier Transform concepts to 2D:
- **Spatial domain** ↔ **Frequency domain** (via FFT)
- **Time** → **X and Y position** in images
- **Frequency** → **Spatial frequency** (cycles per pixel)

**Why log scale?** Magnitude spectrum has a huge dynamic range (DC component is much larger than other frequencies). Log transform: `log(1 + magnitude)` compresses the range for better visualization.

### What to Look For:
- ✅ Bright center = good DC component
- ✅ Radial symmetry = isotropic image features
- ✅ Cross pattern = horizontal/vertical edges in image
- ⚠️ Strong single-frequency lines = periodic noise

---

In [ ]:
# FFT Visualization
if mri_image is not None:
    print('🔬 Computing FFT spectrum...')
    F_shift, mag_spectrum = ts.compute_fft_spectrum(mri_image)
    ts.visualize_frequency_spectrum(mri_image, F_shift)
    plt.show()
    print('✅ FFT analysis complete!')
else:
    print('❌ Please upload an MRI image first!')

---
## 🎯 Step 4: Tumor Detection

Run all segmentation methods and visualize results.

### Methods Compared

#### 1. **Otsu Baseline** (Spatial Domain)
- **Method**: Automatic thresholding based on intensity histogram
- **Principle**: Maximizes inter-class variance between foreground/background
- **Strengths**: Simple, fast, works well on high-contrast images
- **Weaknesses**: Struggles with noise, varying illumination, complex textures

#### 2. **FFT High-Pass** (Frequency Domain)
- **Method**: Removes low frequencies, keeps high frequencies
- **Principle**: Tumor boundaries have high-frequency components
- **Filter Design**: Circular mask with cutoff radius = 25 pixels
- **Strengths**: Emphasizes edges, reduces background influence
- **Weaknesses**: Sensitive to noise, may miss uniform tumor regions

#### 3. **FFT Band-Pass** (Frequency Domain)
- **Method**: Keeps only mid-range frequencies (10-50 pixels)
- **Principle**: Tumor texture falls in mid-frequency range
- **Filter Design**: Ring-shaped mask (inner radius=10, outer radius=50)
- **Strengths**: Captures texture patterns, reduces both noise and background
- **Weaknesses**: May miss tumors without distinct texture

#### 4. **FFT Combined**
- **Method**: Logical OR of High-Pass and Band-Pass results
- **Principle**: Combines edge and texture information
- **Strengths**: More comprehensive detection
- **Weaknesses**: May increase false positives

#### 5. **Blob Detection** (Spatial Domain)
- **Method**: Finds bright, compact regions with shape constraints
- **Principle**: Tumors often appear as bright masses
- **Constraints**: Size (100-25% of image), circularity (>0.12), solidity (>0.65)
- **Strengths**: Robust to edges, focuses on tumor mass
- **Weaknesses**: Misses irregular or ring-shaped tumors

#### 6. **Hybrid Combined** (Ensemble)
- **Method**: Weighted voting (FFT: 1.5 votes, Blob: 1.0 vote)
- **Principle**: Ensemble methods combine multiple approaches
- **Threshold**: Requires ≥1.0 vote (any single method can trigger)
- **Strengths**: Balanced detection, reduces false negatives
- **Weaknesses**: More complex, may over-detect

---

### Confidence Tiers Explained

Each detection is scored based on three factors:
1. **Size** (30%): Is the detected region a reasonable tumor size (0.4-12% of image)?
2. **Shape** (30%): Does it have tumor-like shape (circularity, solidity)?
3. **Contrast** (40%): Is it significantly brighter than background?

**Confidence Thresholds:**
- 🔴 **HIGH (≥65%)**: Strong evidence of tumor, urgent follow-up recommended
- 🟡 **MODERATE (35-65%)**: Suspicious finding, additional imaging advised
- 🟠 **LOW (20-35%)**: Possible abnormality, monitor and re-evaluate
- 🟢 **NONE (<20%)**: No significant tumor detected

### Understanding the Visualization

**Green Cross (✚)**: Marks tumor center using the **centroid of the largest connected component**. This is more robust than using the brightest pixel or mean of all detected pixels, as it represents the geometric center of the detected tumor mass.

**Red Overlay**: Semi-transparent visualization of the detected tumor region (binary mask)

---

In [ ]:
# Run all detection methods
if mri_image is not None:
    print('🎯 Running tumor detection...')

    sensitivity = 0.5

    # Get all segmentations
    results = {
        'Otsu Baseline': otsu_baseline(mri_image),
        'FFT High-Pass': fft_highpass(mri_image, sensitivity),
        'FFT Band-Pass': fft_bandpass(mri_image, sensitivity),
        'FFT Combined': fft_combined(mri_image, sensitivity),
        'Blob Detection': blob_detection(mri_image, sensitivity),
        'Hybrid Combined': hybrid_combined(mri_image, sensitivity)
    }

    # Create visualization
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.ravel()

    # Original image
    axes[0].imshow(mri_image, cmap='gray')
    axes[0].set_title('Original MRI', fontsize=13, fontweight='bold')
    axes[0].axis('off')

    # Each method
    for idx, (method_name, seg_mask) in enumerate(results.items(), start=1):
        confidence, tier = calculate_confidence(mri_image, seg_mask)
        center = get_tumor_center(seg_mask)
        area_pct = (seg_mask.sum() / seg_mask.size) * 100

        axes[idx].imshow(mri_image, cmap='gray')
        axes[idx].imshow(seg_mask, cmap='Reds', alpha=0.5)

        # Mark tumor center
        if center is not None:
            r, c = center
            axes[idx].plot(c, r, 'g+', markersize=20, markeredgewidth=3)

        # Status text
        if tier == 'HIGH':
            status = f'🔴 HIGH\n{confidence*100:.1f}% | {area_pct:.1f}%'
        elif tier == 'MODERATE':
            status = f'🟡 MODERATE\n{confidence*100:.1f}% | {area_pct:.1f}%'
        elif tier == 'LOW':
            status = f'🟠 LOW\n{confidence*100:.1f}% | {area_pct:.1f}%'
        else:
            status = '🟢 Not detected'

        axes[idx].set_title(f'{method_name}\n{status}', fontsize=11, fontweight='bold')
        axes[idx].axis('off')

    # Hide last empty subplot
    axes[7].axis('off')

    plt.suptitle('Tumor Segmentation - All Methods', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

    # Print summary table
    print('\n' + '='*80)
    print(f'{"Method":<20} {"Confidence":<15} {"Tier":<12} {"Area %":<12}')
    print('='*80)
    for method_name, seg_mask in results.items():
        confidence, tier = calculate_confidence(mri_image, seg_mask)
        area_pct = (seg_mask.sum() / seg_mask.size) * 100

        conf_str = f'{confidence*100:.1f}%'
        area_str = f'{area_pct:.2f}%' if tier != 'NONE' else '-'

        print(f'{method_name:<20} {conf_str:<15} {tier:<12} {area_str:<12}')
    print('='*80)

else:
    print('❌ Please upload an MRI image first!')

---
## 📊 Step 5: Quantitative Evaluation (Only if Ground Truth Provided)

Compare all methods using Dice, IoU, and Boundary Accuracy metrics.

### Evaluation Metrics Explained

#### 1. **Dice Similarity Coefficient (DSC)**
**Formula**: `DSC = 2 × |A ∩ B| / (|A| + |B|)`

where:
- `A` = Predicted tumor mask
- `B` = Ground truth mask
- `|A ∩ B|` = Intersection (overlap)
- `|A| + |B|` = Sum of areas

**Range**: [0, 1] where 1 = perfect overlap

**Interpretation:**
- **>0.90**: Excellent segmentation
- **0.80-0.90**: Good segmentation
- **0.70-0.80**: Moderate segmentation
- **<0.70**: Poor segmentation

**Why use Dice?** Most common metric in medical image segmentation. More intuitive than IoU for clinical interpretation. Gives more weight to overlap than non-overlap.

---

#### 2. **Intersection over Union (IoU / Jaccard Index)**
**Formula**: `IoU = |A ∩ B| / |A ∪ B|`

where:
- `|A ∪ B|` = Union (all detected or true tumor pixels)

**Range**: [0, 1] where 1 = perfect overlap

**Interpretation:**
- **>0.75**: Excellent
- **0.50-0.75**: Good
- **<0.50**: Poor

**Relationship to Dice:**
- IoU is **more strict** than Dice
- `Dice = 2×IoU / (1 + IoU)`
- IoU penalizes both false positives AND false negatives more heavily

---

#### 3. **Boundary Accuracy**
**Definition**: Percentage of predicted boundary pixels within `tolerance` distance of true boundary

**Tolerance**: 2 pixels (~2-4mm in typical MRI)

**Range**: [0, 1] where 1 = all boundaries accurate

**Clinical Importance**:
- **Surgical planning** requires precise tumor margins
- **Radiotherapy** needs accurate boundaries to spare healthy tissue
- **2-pixel tolerance** is clinically reasonable (sub-5mm accuracy)

**Interpretation:**
- **>0.90**: Excellent boundary delineation
- **0.75-0.90**: Good, acceptable for most clinical uses
- **<0.75**: May need manual correction

---

### Understanding the Bar Plot

**Color Coding:**
- **Gray**: Otsu Baseline (spatial domain reference)
- **Steel Blue**: FFT methods (frequency domain)
- **Orange**: Blob Detection (spatial domain)
- **Dark Red**: Hybrid Combined (ensemble)

**Three Bars Per Method:**
- Darker = Dice (most common metric)
- Medium = IoU (more strict)
- Lighter = Boundary Accuracy (clinical importance)

**What to Look For:**
- Do FFT methods outperform Otsu?
- Which metric shows the biggest improvement?
- Is the hybrid method the best overall?

---

### Project Hypothesis

**Hypothesis**: FFT-based methods (High-Pass, Band-Pass) will outperform traditional Otsu thresholding for tumor segmentation due to better handling of:
1. Complex texture patterns
2. Varying contrast levels
3. Edge preservation

**Expected Results**: FFT methods should show ~5-20% improvement in Dice coefficient compared to Otsu baseline, especially on high-contrast tumors with clear boundaries.

---

In [ ]:
# Quantitative evaluation
if mri_image is not None and ground_truth_mask is not None:
    print('📊 Calculating quantitative metrics...')

    # Evaluate all methods
    metrics_df, method_masks = evaluate_all_methods(mri_image, ground_truth_mask, sensitivity=0.5)

    # Display table
    print('\n' + '='*80)
    print('QUANTITATIVE EVALUATION RESULTS')
    print('='*80)
    print(metrics_df.to_string(index=False))
    print('='*80)

    # Create bar plot comparison
    fig, ax = plt.subplots(1, 1, figsize=(14, 6))

    methods = metrics_df['Method'].values
    x = np.arange(len(methods))
    width = 0.25

    # Colors: Otsu in gray, FFT methods in blue, Hybrid in red
    colors = ['gray', 'steelblue', 'steelblue', 'steelblue', 'orange', 'darkred']

    # Plot bars for each metric
    bars1 = ax.bar(x - width, metrics_df['Dice'], width, label='Dice Coefficient', color=colors, alpha=0.8)
    bars2 = ax.bar(x, metrics_df['IoU'], width, label='IoU Score', color=colors, alpha=0.6)
    bars3 = ax.bar(x + width, metrics_df['Boundary Accuracy'], width, label='Boundary Accuracy', color=colors, alpha=0.4)

    # Add value labels on bars
    def add_value_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.3f}',
                   ha='center', va='bottom', fontsize=8)

    add_value_labels(bars1)
    add_value_labels(bars2)
    add_value_labels(bars3)

    ax.set_ylabel('Score', fontsize=12, fontweight='bold')
    ax.set_title('Segmentation Performance Comparison\n(Higher is Better)', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(methods, rotation=15, ha='right')
    ax.legend(loc='upper left', fontsize=10)
    ax.set_ylim(0, 1.1)
    ax.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Analysis
    print('\n📈 ANALYSIS:')
    best_dice = metrics_df.loc[metrics_df['Dice'].idxmax()]
    best_iou = metrics_df.loc[metrics_df['IoU'].idxmax()]
    best_boundary = metrics_df.loc[metrics_df['Boundary Accuracy'].idxmax()]

    print(f'   Best Dice: {best_dice["Method"]} ({best_dice["Dice"]:.3f})')
    print(f'   Best IoU: {best_iou["Method"]} ({best_iou["IoU"]:.3f})')
    print(f'   Best Boundary: {best_boundary["Method"]} ({best_boundary["Boundary Accuracy"]:.3f})')

    # Compare to Otsu baseline
    otsu_dice = metrics_df[metrics_df['Method'] == 'Otsu Baseline']['Dice'].values[0]
    print(f'\n   Otsu Baseline Dice: {otsu_dice:.3f}')

    improvements = []
    for _, row in metrics_df.iterrows():
        if row['Method'] != 'Otsu Baseline' and row['Dice'] > otsu_dice:
            improvement = ((row['Dice'] - otsu_dice) / otsu_dice) * 100
            improvements.append(f"   - {row['Method']}: +{improvement:.1f}% improvement")

    if improvements:
        print('\n   Methods outperforming Otsu:')
        for imp in improvements:
            print(imp)
    else:
        print('\n   ⚠️  No method significantly outperformed Otsu baseline')

elif mri_image is not None:
    print('\nℹ️  Skipping quantitative evaluation (no ground truth mask provided)')
else:
    print('❌ Please upload an MRI image first!')

---
## 🏥 Step 6: Final Clinical Report

Summary report with the Hybrid Combined method (recommended).

### Why Hybrid Combined?

The **Hybrid Combined** method is our recommended approach because it:
- Combines frequency-domain analysis (FFT) with spatial-domain blob detection
- Uses weighted voting to balance sensitivity and specificity
- Reduces false negatives (missing tumors) while controlling false positives
- Provides robust detection across various tumor types and imaging conditions

### Clinical Decision Support

This report translates quantitative metrics into **actionable clinical recommendations**:

**🔴 HIGH CONFIDENCE (≥65%)**
- **Interpretation**: Strong evidence of tumor presence
- **Recommendation**: Urgent follow-up with neurology/neurosurgery
- **Next steps**: Confirmatory imaging, biopsy planning, treatment consultation

**🟡 MODERATE CONFIDENCE (35-65%)**
- **Interpretation**: Suspicious finding requiring further investigation
- **Recommendation**: Additional imaging (contrast-enhanced MRI, MR spectroscopy)
- **Next steps**: Repeat scan in 3-6 months, compare with previous studies

**🟠 LOW CONFIDENCE (20-35%)**
- **Interpretation**: Possible abnormality, uncertain significance
- **Recommendation**: Monitor with routine follow-up
- **Next steps**: Correlate with clinical symptoms, consider repeat imaging in 6-12 months

**🟢 NO TUMOR DETECTED (<20%)**
- **Interpretation**: No significant abnormality detected
- **Recommendation**: No immediate action required
- **Next steps**: Routine screening based on patient risk factors

### Important Disclaimers

⚠️ **This is a demonstration tool for educational purposes (BME 271D Final Project)**  
⚠️ **Not validated for clinical use** - would require FDA clearance  
⚠️ **Not a replacement for radiologist expertise** - intended to assist, not replace  
⚠️ **Performance varies** based on image quality, tumor type, and imaging parameters

### Real-World Deployment Considerations

To use this system clinically, we would need:
1. **Extensive validation** on diverse patient datasets
2. **Performance benchmarking** against expert radiologists
3. **Sensitivity/specificity optimization** for specific clinical workflows
4. **Integration** with PACS (Picture Archiving and Communication System)
5. **Regulatory approval** (FDA, CE marking)
6. **Continuous monitoring** of real-world performance

---

In [ ]:
# Final clinical report
if mri_image is not None and 'results' in dir():
    print('🏥 Generating final clinical report...')

    final_mask = results['Hybrid Combined']
    confidence, tier = calculate_confidence(mri_image, final_mask)
    center = get_tumor_center(final_mask)
    area_pct = (final_mask.sum() / final_mask.size) * 100

    # Visualization
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))

    # Original
    axes[0].imshow(mri_image, cmap='gray')
    axes[0].set_title('Original MRI', fontsize=14, fontweight='bold')
    axes[0].axis('off')

    # Detection
    axes[1].imshow(mri_image, cmap='gray')
    if tier != 'NONE':
        axes[1].imshow(final_mask, cmap='Reds', alpha=0.5)
        if center is not None:
            r, c = center
            axes[1].plot(c, r, 'g+', markersize=30, markeredgewidth=4)
    axes[1].set_title('Hybrid Detection Result', fontsize=14, fontweight='bold')
    axes[1].axis('off')

    # Report
    axes[2].axis('off')

    if tier == 'HIGH':
        status = '⚠️ TUMOR DETECTED'
        verdict = '🔴 HIGH CONFIDENCE'
        recommendation = 'Urgent follow-up recommended'
    elif tier == 'MODERATE':
        status = '❓ SUSPICIOUS FINDING'
        verdict = '🟡 MODERATE CONFIDENCE'
        recommendation = 'Additional imaging advised'
    elif tier == 'LOW':
        status = '❓ POSSIBLE ABNORMALITY'
        verdict = '🟠 LOW CONFIDENCE'
        recommendation = 'Monitor and re-evaluate'
    else:
        status = '✅ NO TUMOR DETECTED'
        verdict = '🟢 Normal Brain'
        recommendation = 'No immediate action required'

    report_text = f"""

    {status}

    ─────────────────────────

    Confidence: {confidence*100:.1f}%
    Affected Area: {area_pct:.2f}%

    Method: Hybrid Ensemble
    (FFT + Blob Detection)

    ─────────────────────────

    {verdict}

    {recommendation}
    """

    axes[2].text(0.05, 0.5, report_text, fontsize=12, verticalalignment='center',
                fontfamily='monospace',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    axes[2].set_title('Clinical Report', fontsize=14, fontweight='bold')

    plt.tight_layout()
    plt.show()

    print('\n' + '='*70)
    print(f'  {verdict}')
    print('='*70)

else:
    print('❌ Please run detection first!')

---
## 🎓 Conclusion & Key Takeaways

### What We Demonstrated

This project successfully applied **Signals & Systems** concepts from BME 271D to a real biomedical engineering problem:

1. **2D Fourier Transform**: Extended 1D FFT to medical images
2. **Filter Design**: Implemented high-pass and band-pass filters in frequency domain
3. **System Analysis**: Compared frequency-domain vs. spatial-domain methods
4. **Performance Evaluation**: Used quantitative metrics (Dice, IoU, Boundary Accuracy)
5. **Ensemble Methods**: Combined multiple approaches for robust detection

### Key Findings

**Strengths of FFT-Based Methods:**
- ✅ Better edge detection (high-pass filtering)
- ✅ Texture analysis capabilities (band-pass filtering)
- ✅ Reduced sensitivity to global intensity variations
- ✅ Complementary information to spatial methods

**Limitations:**
- ⚠️ Performance depends on tumor contrast
- ⚠️ Sensitive to image quality and noise
- ⚠️ Requires parameter tuning (cutoff frequencies)
- ⚠️ Computational cost of FFT operations

### Clinical Relevance

**Potential Applications:**
- Computer-aided detection (CAD) systems
- Radiologist decision support
- Treatment planning assistance
- Automated screening workflows

**Impact:**
- Reduces radiologist workload
- Standardizes segmentation quality
- Enables faster diagnosis
- Improves patient outcomes through earlier detection

### Future Improvements

1. **Adaptive Filtering**: Automatically adjust parameters based on image characteristics
2. **3D Analysis**: Process entire MRI volumes instead of 2D slices
3. **Multi-Modal Fusion**: Combine T1, T2, FLAIR sequences
4. **Deep Learning Integration**: Use FFT features as input to neural networks
5. **Real-Time Performance**: GPU acceleration for clinical workflow
6. **Uncertainty Quantification**: Provide confidence intervals for measurements

### Course Connection

This project demonstrates how fundamental signal processing concepts from BME 271D apply to real-world biomedical problems:

- **Fourier Transform** → Analyzing medical images in frequency domain
- **Filter Design** → Enhancing tumor features
- **System Analysis** → Comparing different detection methods
- **Performance Metrics** → Quantifying segmentation quality

### Acknowledgments

- BME 271D teaching staff for course content
- Duke University BME Department
- Open-source medical imaging datasets
- Python scientific computing community

---

## 📚 References

1. Otsu, N. (1979). "A threshold selection method from gray-level histograms." *IEEE Trans. Systems, Man, and Cybernetics*, 9(1), 62-66.

2. Dice, L.R. (1945). "Measures of the amount of ecologic association between species." *Ecology*, 26(3), 297-302.

3. Menze, B.H., et al. (2015). "The Multimodal Brain Tumor Image Segmentation Benchmark (BRATS)." *IEEE Transactions on Medical Imaging*, 34(10), 1993-2024.

4. Gonzalez, R.C., & Woods, R.E. (2018). *Digital Image Processing* (4th ed.). Pearson.

5. The Cancer Imaging Archive (TCIA): https://www.cancerimagingarchive.net/

---

*BME 271D: Signals and Systems in Biomedical Engineering*  
*Duke University, Fall 2024*  
*Team: Ege Özemek, Max Bazan, Sasha Nikiforov*

---